In [4]:
from gensim import corpora, models
#jieba.posseg是分词器  ”我爱北京天安门“会变成我 r    爱 v    北京 ns    天安门 ns
import jieba.posseg as jp, jieba
import reader
import numpy as np
# 文本集
# file_name = 'select_description_from_sys_district.csv'
file_name = 'hotel_description_small.csv'
if reader.check_file(file_name, url=''):
    hotelsDesc = reader.read_data_as_array(file_name)
else:
    quit()
texts = hotelsDesc # 生成格式如下
# print(hotelsDesc)
# texts = [
#     '美国教练坦言，没输给中国女排，是输给了郎平',
#     '美国无缘四强，听听主教练的评价',
#     '中国女排晋级世锦赛四强，全面解析主教练郎平的执教艺术',
#     '为什么越来越多的人买MPV，而放弃SUV？跑一趟长途就知道了',
#     '跑了长途才知道，SUV和轿车之间的差距',
#     '家用的轿车买什么好']
# 分词过滤条件
# jieba.add_word('一方天地', 9, 'n')
# n	普通名词	f	方位名词	s	处所名词	t	时间名词
# nr	人名	ns	地名	nt	机构团体名	nw	作品名
# nz	其他专名	v	普通动词	vd	动副词	vn	名动词
# a	形容词	ad	副形词	an	名形词	d	副词
# m	数量词	q	量词	r	代词	p	介词
# c	连词	u	助词	xc	其他虚词	w	
# PER	人名	LOC	地名	ORG	机构名	TIME	时间
flags = ('n', 'f', 's', 't', 'nr', 'ns', 'nt', 'nw', 'nz','v','vd','vn','a','ad','an','d','q','r','p','c','u','xc','PER','LOC','ORG')  # 词性
stop_file_name = 'stopwords.txt'
if reader.check_file(stop_file_name, url=''):
    stopWords = reader.read_data_as_array(stop_file_name)
else:
    quit()

stopword_list = set(stopWords)  # 停词。格式为{'', '她', '无', '至', '前后', '即令', '大', '往', '不特', '有的', '替代',。。。}


文件  hotel_description_small.csv 存在。盛宴大幕拉开了。
段落处理完毕
文件  stopwords.txt 存在。盛宴大幕拉开了。
段落处理完毕


In [5]:
# 分词
words_ls = []
words_speech = []
# 将texts每个item的句子拆成单词
# jieba.enable_paddle() #启动paddle模式
whole_words = []
for text in texts:
    temp_words = [w.word for w in jp.cut(text,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
    temp_speechs = [w.flag for w in jp.cut(text,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0]
#     words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stopwords and len(w.word) > 0]
    print('---{}, {}',temp_words, len(temp_words))
    print('---{}, {}',temp_speechs, len(temp_speechs))
    if len(temp_words) > 0:
        words_ls.append(temp_words)
        words_speech.append(temp_speechs)
        whole_words.extend(temp_words)
print('++++++++{}', whole_words)
# for w in jp.cut(text,use_paddle=True):
#天安门 ns  其中天安门用w.word   ns用w.flag
#     print('---{}',w)
# 构造词典 这个词典是将拼接后的文本放进去，每项都是唯一的。试验下来可以去重
dictionary = corpora.Dictionary(words_ls)
# for i in dictionary:
#     print('_+_+_+_+_+_+{}', i)
# 基于词典，使【词】→【稀疏向量】，并将向量放入列表，形成【稀疏向量集】
#这里就是创建每个句子的词组的词频
corpus = [dictionary.doc2bow(words) for words in words_ls]
# print(corpus)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.976 seconds.
Prefix dict has been built successfully.


---{}, {} ['开业', '间房', '上海', '时生隅', '园林', '文化', '酒店', '秋末', '时生隅', '初见', '心愿', '想要', '快节奏', '都市', '讲诉', '时光', '生命', '角落', '对错', '闻', '浮沉', '便是', '时生隅', '由来', '秉承', '初心', '川沙', '地铁站', '商圈', '觅得', '天地', '充满', '江南', '气息', '园林', '叶园', '长廊', '湖景', '古雅', '庭院', '大气', '快节奏', '民俗文化', '交融', '幽静', '明快', '小憩', '居', '别样', '天地', '时生隅', '新老朋友', '叶园', '期待', '共', '当下', '时光', '言欢'] 58
---{}, {} ['n', 'n', 'ns', 'n', 'n', 'n', 'n', 't', 'n', 'v', 'n', 'v', 'n', 'ns', 'v', 'n', 'vn', 'n', 'v', 'v', 'a', 'v', 'n', 'c', 'v', 'n', 'ns', 'n', 'n', 'v', 'n', 'a', 'ns', 'n', 'n', 'nr', 'n', 'n', 'nr', 'n', 'n', 'n', 'n', 'n', 'a', 'a', 'nr', 'v', 'r', 'n', 'n', 'n', 'nr', 'v', 'd', 't', 'n', 'nr'] 58
---{}, {} ['开业', '装修', '间', '房云', '夜泊', '酒店', '上海浦东国际机场', '店', '位于', '浦东新区', '祝桥秋亭', '路', '毗邻', '上海浦东国际机场', '车程', '分钟', '上海', '迪士尼', '分钟', '车程', '上海', '动物园', '分钟', '车程', '酒店', '地理位置', '优越', '配备', '大巴', '中巴', '豪华轿车', '接送', '客户', '酒店', '拥有', '房间', '床', '间', '行政', '大床', '间', '标间', '间亲', '子房', '间'] 45
---{}, {} ['n', '

In [ ]:
# lda模型，num_topics设置主题的个数
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
# 打印所有主题，每个主题显示5个词
for topic in lda.print_topics(num_words=10, num_topics=10):
    print(topic)
# 主题推断
# print(lda.inference(corpus))

In [ ]:
for e, values in enumerate(lda.inference(corpus)[0]):
    print(texts[e])
    for ee, value in enumerate(values):
        print('\t主题%d推断值%.2f' % (ee, value))

In [ ]:
text5 = '2013年开业74间房。海友酒店（上海德平路地铁站店）位于浦东新区德平路，毗邻上海八佰伴、陆家嘴、世纪公园、金桥国际商业中心。酒店附近多部公交与地铁可通往人民广场、外滩、南京路步行街等繁华商业区及旅游景点，交通便捷。 　　海友酒店（上海德平路地铁站店）拥有大床房、家庭房、双床房等多种房型供您选择，可满足顾客不同的入住需求。客房内配有24小时热水、液晶电视机等完善的配套设施，温馨舒适。 　　海友客栈（上海德平路店）竭诚为您提供至善至美的上佳服务，欢迎您随时光临下榻。'
# bow = dictionary.doc2bow([word.word for word in jp.cut(text5) if word.flag in flags and word.word not in stopwords])
bow = dictionary.doc2bow([w.word for w in jp.cut(text,use_paddle=True) if w.flag in flags and w.word not in stopWords and len(w.word) > 0])
ndarray = lda.inference([bow])[0]
print(text5)
for e, value in enumerate(ndarray[0]):
    print('\t主题%d推断值%.2f' % (e, value))

In [ ]:
word_id = dictionary.doc2idx(['开业'])[0]
print(dictionary.doc2idx(['开业'])[0])
for i in lda.get_term_topics(word_id):
    print('【开业】与【主题%d】的关系值：%.2f%%' % (i[0], i[1]*100))

In [ ]:
for word, word_id in dictionary.token2id.items():
    print(word, lda.get_term_topics(word_id, minimum_probability=1e-8))

In [ ]:
print(lda.show_topic(0, 9999))
print('概率总和', sum(i[1] for i in lda.show_topic(0, 9999)))

In [ ]:
vocab = dictionary.token2id
print(vocab)
print ('{} unique characters'.format(len(vocab)))

In [ ]:
import numpy as np

char2idx = {u:i for i, u in enumerate(vocab)}
# print(char2idx)
idx2char = {u:i for u, i in enumerate(vocab)}
print(char2idx)

In [ ]:
import tensorflow as tf
# 设定每个输入句子长度的最大值
seq_length = 60
examples_per_epoch = len(whole_words)//seq_length
text_as_int = np.array([char2idx[c] for c in whole_words])
print(text_as_int)
# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

In [ ]:
words = sorted(list(set(whole_words)))
word_indices = dict((word, words.index(word)) for word in words)
print(words)
print(word_indices)

maxlen = 30
sentences = []
next_word = []

for i in range(0, len(whole_words) - maxlen):
    sentences.append(whole_words[i: i + maxlen])
    next_word.append(whole_words[i + maxlen])
print('提取的句子总数:', len(sentences))

x = np.zeros((len(sentences), maxlen), dtype='float32') # Embedding的输入是2维张量（句子数，序列长度）
y = np.zeros((len(sentences)), dtype='float32')
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t] = word_indices[word]
    y[i] = word_indices[next_word[i]]

In [ ]:
import sys
print(np.round((sys.getsizeof(x) / 1024 / 1024 / 1024), 4), "GB") 
print(x.shape, y.shape)

In [ ]:
import tensorflow as tf
import os
main_input = tf.keras.layers.Input(shape=(maxlen, ), dtype='float32') 
model_1 = tf.keras.layers.Embedding(len(words), maxlen, input_length=maxlen)(main_input)
model_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(maxlen, return_sequences=True))(model_1)
model_1 = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(maxlen))(model_1)
output = tf.keras.layers.Dense(len(words), activation='softmax')(model_1)  
model = tf.keras.models.Model(main_input, output)
# optimizer = tf.keras.optimizers.RMSprop(lr=3e-3)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# 检查点保存至的目录
checkpoint_dir = './ldatraining_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.summary()
history = model.fit(x, y, epochs=10, batch_size=1024, callbacks=[checkpoint_callback])

In [ ]:
def generate_text(model, start_string):
    num_generate = 100
    input_eval = [char2idx[s.word] for s in jp.cut(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)
    # 空字符串用于存储结果
    text_generated = []

    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0

    # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        print(predictions)
        # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
        print(predictions)
        # 用分类分布预测模型返回的字符
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"酒店服务"))